In [1]:
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import copy
import ast
import attributes

from ipywidgets import interact, Dropdown
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

#pd.set_option('display.max_rows', 200)

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_HarmonizeV3.xlsx"
SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie.xlsx"

In [3]:
xl = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, None);
df_table = xl['Objecten']
df_table.head(30)

,Unnamed: 0,Object,Tabellen,Overerven,Samenvoegen,ABR-materiaal,ABR-submateriaal,Unnamed: 7
0,0.0,Ignore,"["".*backup.*"", "".*kopie.*"", "".*tijdelijk.*"", ""...",NaN,NaN,NaN,NaN,NaN
1,1.0,Tekening,"[""^TEKENING.*""]",NaN,NaN,NaN,NaN,NaN
2,2.0,Vondst,"[""^VONDSTENLIJST"", ""^VONDST$"", ""^VELDVONDST"", ...",NaN,NaN,NaN,NaN,NaN
3,3.0,Spoor,"[""^SPOREN$"", ""^SPOOR$"", ""sporen""]",NaN,NaN,NaN,NaN,NaN
4,4.0,Vulling,"[""^VULLING.*"", ""MUUR""]",NaN,NaN,NaN,NaN,NaN
5,5.0,Dia,"[""DIA.*""]",NaN,NaN,NaN,NaN,NaN
6,6.0,Foto,"["".*FOTO.*""]",NaN,NaN,NaN,NaN,NaN
7,27.0,Fotobeschrijving,"[""DIAOPGRAVING"", ""DIAVOORWERP"", ""Fotolijst""]",NaN,NaN,NaN,NaN,NaN
8,28.0,Fotokoppel,"[""Foto Totaal Tabel""]",NaN,NaN,NaN,NaN,NaN
9,7.0,Put,"[""^PUT$"", ""PUTTEN""]",NaN,NaN,NaN,NaN,NaN


## Find all Attributes and connect them to the Objects

In [4]:

df = attributes.getAllAttributes()
df = df.sort_values(by=['Object', 'count'], ascending=False)
df[df['Object'] == 'Tekening']

len(df)

2633

In [5]:
lst = df['Object'].unique()
Objecten_lst = Dropdown(options = sorted(lst))

@interact(obj = Objecten_lst)
def print_city(obj):
    display(df[df.Object == obj].style.apply(lambda x: ["color: green" if bool(set(x.projecten) & set(['DC179', 'DC154'])) else "" for v in x], axis = 1))


interactive(children=(Dropdown(description='obj', options=('Aardewerk', 'Artefact', 'Bouwaardewerk', 'DT_Soort…

In [6]:
def setNr(kolom):
    string = str(kolom)
    return string[:len(string)-6] + 'nr' if string.endswith('nummer') else string

df["sugg_kolom"] = df["Kolommen"].str.lower()
#df["sugg_kolom"] = df.apply(lambda x: setNr(df["sugg_kolom"]), axis=1)
df["sugg_omschr"] = df.apply(lambda x: x['omschrijvingen'][0].split()[0].lower() if len(x['omschrijvingen']) > 0 else "", axis=1)
df["sugg_omschr"] = df.apply(lambda x: setNr(x['sugg_omschr']), axis=1)
df["new_attr"] = ""
df["new_attr_artefact"] = ""
df.head(30)

,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld,sugg_kolom,sugg_omschr,new_attr,new_attr_artefact
2601,Vulling,KLEUR,"[kleur, kleur (volgens standaard codering), kl...",186,[DC024],kleur,5723.0,50.0,kleur,kleur,,
2614,Vulling,OPMERKING,"[extra opmerkingen, eventuele extra opmerkingen]",164,[DC024],opmerkingen,5816.0,51.0,opmerking,extra,,
2621,Vulling,SPOOR,[spoornummer],163,[DC024],spoornr,5220.0,46.0,spoor,spoornr,,
2580,Vulling,(none),[TOTAALTABEL VULLINGEN],124,[],,NaN,NaN,(none),totaaltabel,,
2581,Vulling,10LGMT,"[10 lagen maat, 10 lagenmaat in cm]",124,[DC024],,3974.0,35.0,10lgmt,10,,
2582,Vulling,BB1,"[steenformaat 1 breedte in cm, formaat steen]",124,[DC024],breedte_baksteen1,3974.0,35.0,bb1,steenformaat,,
2583,Vulling,BB2,"[steenformaat 2 breedte in cm, formaat steen]",124,[DC024],breedte_baksteen2,3974.0,35.0,bb2,steenformaat,,
2584,Vulling,BB3,"[steenformaat 3 breedte in cm, formaat steen]",124,[DC024],breedte_baksteen3,3974.0,35.0,bb3,steenformaat,,
2586,Vulling,BOUWMAT,"[type bouwmateriaal, bouwmateriaal]",124,[DC024],,3974.0,35.0,bouwmat,type,,
2592,Vulling,HB1,"[steenformaat 1 hoogte in cm, formaat steen]",124,[DC024],hoogte_baksteen1,3974.0,35.0,hb1,steenformaat,,


In [7]:
lst_objects = df['Object'].unique()

with pd.ExcelWriter(SUGGESTIE_XL) as writer:
    for obj in lst_objects:
        df[df.Object == obj].to_excel(writer, sheet_name=obj)
    writer.save()


/tmp/ipykernel_23946/1422136754.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [8]:
lst = list(df_table[df_table.Overerven == 'Artefact']['Object'])
lst.sort()
#lst = lst.sort()
for tp in lst:
    print(tp + ' = \"' + tp + '\"')

Aardewerk = "Aardewerk"
Bouwaardewerk = "Bouwaardewerk"
Dierlijk_Bot = "Dierlijk_Bot"
Glas = "Glas"
Hout = "Hout"
Kleipijp = "Kleipijp"
Leer = "Leer"
Menselijk_Bot = "Menselijk_Bot"
Metaal = "Metaal"
Munt = "Munt"
Onbekend = "Onbekend"
Schelp = "Schelp"
Steen = "Steen"
Textiel = "Textiel"


In [9]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)

with engine.connect() as connection:
    df = pd.read_sql_query('SELECT brondata, artefactsoort from "Def_Artefact"', connection)

list(df['artefactsoort'].unique())

ObjectNotExecutableError: Not an executable object: 'SELECT brondata, artefactsoort from "Def_Artefact"'

In [ ]:
lst